In [1]:
from shapely.geometry import Polygon, box
import geopandas as gpd
import fiona
from shapely.ops import unary_union
from fiona.crs import from_epsg
import json
import rasterio
from tqdm import tqdm

In [2]:
tiff_path = '../TropicalTreeCover_NorthernQLD-0000000000-0000000000.tif'
path = "bounding_boxes.geojson"
with open(path) as file:
    data = json.load(file)


In [3]:
len(data['features'])

16375

In [20]:
coords = data['features'][1020]['geometry']['coordinates'][0]
bl = (min(row[0] for row in coords), min(row[1] for row in coords))
tr = (max(row[0] for row in coords), max(row[1] for row in coords))
bounds = (*bl, *tr)

In [24]:
bounds = box(*bounds)

In [29]:
bounds.bounds

(121.91825838609505,
 -14.048010970322123,
 121.96423690097433,
 -14.002032455442848)

In [ ]:
images = []
with rasterio.open(tiff_path) as src:
        
    with open(path) as file:
        data = json.load(file)
        
    for i in tqdm(range(len(data['features']))):
        coords = data['features'][i]['geometry']['coordinates'][0]
        bl = (min(row[0] for row in coords), min(row[1] for row in coords))
        tr = (max(row[0] for row in coords), max(row[1] for row in coords))
        bounds = (*bl, *tr)
        bounds = box(*bounds)
        min_x = bounds.bounds[0] 
        min_y = bounds.bounds[1] 
        max_x = bounds.bounds[2]
        max_y = bounds.bounds[3]

        window = src.window(min_x, min_y, max_x, max_y)
        image_array = src.read(window=window)

        images.append(image_array)


 97%|██████████████████████████████████████████████████████████████████████████▍  | 15838/16375 [02:17<16:45,  1.87s/it]